# Clasificación de frutas frescas y podridas con ResNet18


Este notebook implementa una solución de clasificación de frutas (frescas y podridas) usando una red neuronal convolucional preentrenada (ResNet18). Utiliza un dataset obtenido desde Kaggle Hub y realiza las siguientes etapas:

1. Descarga y carga de datos.
2. Transformaciones y preprocesamiento.
3. Definición y ajuste del modelo.
4. Entrenamiento y validación.
5. Visualización de métricas.
6. Guardado del modelo entrenado.
7. Inferencia sobre una imagen de prueba.


In [ ]:
# Librerías
import kagglehub
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from google.colab import drive
from PIL import Image  # Para manejo de imágenes

print("✅ Librerías importadas correctamente")

# Descarga del dataset desde Kaggle Hub
try:
    path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")
    print("✅ Dataset descargado correctamente")
    print("Path to dataset files:", path)

    # Configuración general
    batch_size = 128
    img_size = 224
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🚀 Dispositivo de ejecución: {device}")

    # Transformaciones para las imágenes
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    print("✅ Configuración y transformaciones definidas")

    # Cargar datasets
    train_data_dir = os.path.join(path, 'dataset', 'train')
    val_data_dir = os.path.join(path, 'dataset', 'test')

    train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
    val_dataset = datasets.ImageFolder(root=val_data_dir, transform=transform)

    class_names = train_dataset.classes
    print(f"🔍 Clases detectadas: {class_names}")
    print(f"📊 Total de imágenes de entrenamiento: {len(train_dataset)}")
    print(f"📊 Total de imágenes de validación: {len(val_dataset)}")

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    print("✅ Dataset cargado y dividido correctamente")

    # Cargar modelo ResNet18 preentrenado
    model = models.resnet18(pretrained=True)
    print("🎯 ResNet18 cargado")

    for param in model.parameters():
        param.requires_grad = False
    print("❄️ Parámetros congelados")

    # Reemplazo de la capa final
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, len(class_names))
    print(f"🔄 Capa FC reemplazada para {len(class_names)} clases")

    model = model.to(device)
    print(f"📌 Modelo movido a {device}")

    # Función de pérdida y optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
    epochs = 10

    print("⚙️ Configuración de entrenamiento definida")

    train_losses = []
    val_losses = []
    val_accuracies = []

    print("🚀 Comenzando entrenamiento...")

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)

        # Validación
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        accuracy = 100 * correct / total
        val_accuracies.append(accuracy)

        print(f"✅ Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {epoch_loss:.4f} - "
              f"Val Loss: {val_loss:.4f} - "
              f"Accuracy: {accuracy:.2f}%")

    print("🎉 Entrenamiento completado!")

    # Visualización de resultados
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Entrenamiento')
    plt.plot(val_losses, label='Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.title('Curva de Pérdida')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(val_accuracies, label='Validación', color='green')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión (%)')
    plt.title('Precisión en Validación')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Guardar modelo
    try:
        model_path = "modelo.pth"
        torch.save(model.state_dict(), model_path)
        print(f"💾 Modelo guardado como: {model_path}")
    except Exception as e:
        print(f"❌ Error al guardar el modelo: {e}")

    # Inferencia
    print("--- Fase de Inferencia ---")
    sample_image_path, _ = val_dataset.samples[0]
    print(f"⏳ Inferencia sobre: {sample_image_path}")

    image = Image.open(sample_image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted_class_index = torch.max(outputs, 1)

    predicted_class_name = class_names[predicted_class_index.item()]
    print(f"✅ La fruta detectada es: {predicted_class_name}")

except Exception as e:
    print(f"❌ Error inesperado: {e}")